# Qamomile v0.8.0

---

This release mayjorly encences the accessibility for broader people by adding some [tutorials in Japanese](https://jij-inc.github.io/Qamomile/ja/), usability by supporting [OMMX v2](https://jij-inc.github.io/ommx/en/introduction.html) and introducing a new target quantum SDK, [CUDA-Q](https://developer.nvidia.com/cuda-q), as well as [Bloqade-Analog](https://github.com/QuEraComputing/bloqade-analog). We believe that, especially OMMX v2 support is a big step for us and users as OMMX allows us to compare quantum optimisation algorithms and classical solvers esaier than before as OMMX provides us the united form of optimisation problems.

We have other updates having slightly smaller impacts as well. Please have a look at our [releases](https://github.com/Jij-Inc/Qamomile/releases) and individual PRs for more details.

## ✨ New Features

### OMMX v2 Support ([#126](https://github.com/Jij-Inc/Qamomile/pull/126), [#198](https://github.com/Jij-Inc/Qamomile/pull/198))

OMMX v2 brings major performance improvements through a new Rust-based implementation, replacing the previous Protocol Buffer-based approach (see [OMMX Python SDK 2.0.0](https://github.com/Jij-Inc/ommx/blob/main/docs/en/release_note/ommx-2.0.0.md) for more details). This update provides:

  - **Enhanced Performance**: Computation complexity improvements, especially for large-scale problems
  - **Improved API**: More intuitive and AI-friendly API design
  - **Better Type Safety**: Rust-level type checking for more robust optimization problem handling

Note that, OMMX v2 does bring performance improvements of handling instances, but it does not mean it brings performance improvements at quantum algorithm side such as a circuit running time.

In [ ]:
# Import necessary libraries.
from collections import defaultdict

import jijmodeling as jm
import matplotlib.pyplot as plt
import numpy as np
import ommx.v1
import qiskit.primitives as qk_pr
from scipy.optimize import minimize

import qamomile.core as qm
import qamomile.qiskit as qm_qk


# Create optimization problem with JijModeling.
Q = jm.Placeholder("Q", ndim=2)
n = Q.len_at(0, latex="n")
x = jm.BinaryVar("x", shape=(n,))

problem = jm.Problem("qubo")
i, j = jm.Element("i", n), jm.Element("j", n)
problem += jm.sum([i, j], Q[i, j] * x[i] * x[j])

# Compile with data, which will produce ommx.v1.Instance.
instance_data = {"Q": [[0.1, 0.2, -0.1], [0.2, 0.3, 0.4], [-0.1, 0.4, 0.0]]}
interpreter = jm.Interpreter(instance_data)
instance: ommx.v1.Instance = interpreter.eval_problem(problem)  # NEW

# Convert to QAOA using Qamomile.
qaoa_converter = qm.qaoa.QAOAConverter(instance)
p = 5
qaoa_circuit = qaoa_converter.get_qaoa_ansatz(p=p)
qaoa_hamiltonian = qaoa_converter.get_cost_hamiltonian()

# Convert Qamomile circuit and Hamiltonian to Qiskit format.
qk_transpiler = qm_qk.QiskitTranspiler()
qk_circuit = qk_transpiler.transpile_circuit(qaoa_circuit)
qk_hamiltonian = qk_transpiler.transpile_hamiltonian(qaoa_hamiltonian)

# Define cost estimation function using Qiskit primitives.
estimator = qk_pr.StatevectorEstimator()
cost_history = []
def estimate_cost(param_values):
    try:
        job = estimator.run([(qk_circuit, qk_hamiltonian, param_values)])
        result = job.result()[0]
        cost = result.data["evs"]
        cost_history.append(cost)
        return cost
    except Exception as e:
        print(f"Error during cost estimation: {e}")
        return np.inf

# Optimise the QAOA parameters using COBYLA.
initial_params = np.random.uniform(low=-np.pi / 4, high=np.pi / 4, size=2 * p)
result = minimize(
    estimate_cost,
    initial_params,
    method="COBYLA",
    options={"maxiter": 2000, "tol": 1e-2},
)

# Run the QAOA circuit with the optimised parameters.
qk_circuit.measure_all()
sampler = qk_pr.StatevectorSampler()
job = sampler.run([(qk_circuit, result.x)], shots=10000)
job_result = job.result()[0]
sampleset: ommx.v1.SampleSet = qaoa_converter.decode(qk_transpiler, job_result.data["meas"])  # NEW

# Create a dictionary to group energies and count their frequencies
energy_freq = defaultdict(int)
for sample_id in sampleset.sample_ids:
    sample = sampleset.get(sample_id)
    energy_freq[sample.objective] += 1
energies = list(energy_freq.keys())
frequencies = list(energy_freq.values())

# Show the cost distribution.
plt.bar(energies, frequencies, width=0.5)
plt.title("cost distribution of feasible solution (No nomarilisation)", fontsize=15)
plt.ylabel("Frequency", fontsize=15)
plt.xlabel("Energy", fontsize=15)
plt.show()


### CUDA-Q Support ([#143](https://github.com/Jij-Inc/Qamomile/pull/143))
CUDA-Q stremlines hybrid application development and promotes productivity and scalability in quantum computing. And now, you can compile your circuits and Hamiltonians in Qamomile format into CUDA-Q format!

In [ ]:
import cudaq

from qamomile.core.circuit import QuantumCircuit as QamomileCircuit
from qamomile.core.operator import Hamiltonian, X, Y, Z
from qamomile.cudaq.transpiler import CudaqTranspiler

# Create an instance of the CudaqTranspiler.
cudaq_transpiler = CudaqTranspiler()

# Define a simple quantum circuit using Qamomile.
qc = QamomileCircuit(1)
qc.h(0)
qc.x(0)

# Transpile the circuit using CudaqTranspiler.
transpiled_circuit = cudaq_transpiler.transpile_circuit(qc)

# Draw the transpiled circuit.
print(type(transpiled_circuit))
print(cudaq.draw(transpiled_circuit, []))

# Create a Hamiltonian using Qamomile.
hamiltonian = Hamiltonian()
hamiltonian += X(0) * Z(1)
hamiltonian += Y(0) * Y(1) * Z(2) * X(3) * X(4)

# Transpile the Hamiltonian using CudaqTranspiler.
transpiled_hamiltonian = cudaq_transpiler.transpile_hamiltonian(hamiltonian)
# Show the transpiled hamiltonian.
print(type(transpiled_hamiltonian))
print(transpiled_hamiltonian)

### Bloqade-Analog

## 🐛 Bug Fixes

See our [releases](https://github.com/Jij-Inc/Qamomile/releases) for details.

## 🛠️ Other Changes and Improvements

See our [releases](https://github.com/Jij-Inc/Qamomile/releases) for details.

## 💬 Feedback
With these new features, Qamomile becomes a powerful tool for enabling users to access several quantum SDKs in a same way and compare the results with classical solvers. Try out Qamomile 0.8.0!


Please submit any feedback or bug reports to [GitHub Issues](https://github.com/Jij-Inc/Qamomile/issues).